## Imports

In [1]:
from ultralytics import YOLO
from PIL import Image
import mediapipe as mp
import numpy as np
import pandas as pd
import cv2
import os
import sys

from scripts import pose_estimator
from scripts.classifier import classify_by_hip_distance

## Setup

In [2]:
model = YOLO(r"E:/GitHub/porsche-students-contest/best.pt")

In [7]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

## Make predictions

In [52]:
source = Image.open(r"E:\GitHub\porsche-students-contest\images\src9.jpg")

In [53]:
results = model.predict(source=source, save=True)


0: 544x800 2 crosswalks, 1 person, 185.0ms
Speed: 1.0ms preprocess, 185.0ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)
Results saved to E:\GitHub\porsche-students-contest\runs\detect\predict6


In [54]:
crosswalks = []
persons = []

In [55]:
for idx, box in enumerate(results[0].boxes.boxes):
    if box[-1] == 1:
        x_min, y_min, x_max, y_max = int(box[0]), int(box[1]), int(box[2]), int(box[3])
        crosswalks.append((x_min, y_min, x_max, y_max))
    elif box[-1] == 2:
        x_min, y_min, x_max, y_max = int(box[0]), int(box[1]), int(box[2]), int(box[3])
        persons.append((x_min, y_min, x_max, y_max))

In [58]:
if len(crosswalks) == 0:
    print("nici un pericol")
elif len(crosswalks) == 1:
    print("got 1 crosswalk")
    for person in persons: 
        if crosswalks[0][0] < person[0] < crosswalks[0][2] and crosswalks[0][1] < person[3] < crosswalks[0][3]:
            print("danger, pe trecere")
        elif crosswalks[0][1] < person[3] + 15 or crosswalks[0][3] > person[3] - 15:
            if person[0] < crosswalks[0][2] + 350:
                # Get orientation :
                landmarks = pose_estimator.get_pose_landmarks(np.asarray(source)[persons[0][1]:persons[0][3], persons[0][0]:persons[0][2]])
                hips_label = classify_by_hip_distance(landmarks)
                if hips_label == "side":
                    print("warning, pe langa trecere")
                else:
                    print("langa trecere, dar nu se uita in stanga sau dreapta")
        else:
            print("nici un pericol")
elif len(crosswalks) > 1:
    for idx, crosswalk in enumerate(crosswalks):
        print(f'crosswalk {idx}:')
        for person in persons: 
            if crosswalk[0] < person[0] < crosswalk[2] and crosswalk[1] < person[3] < crosswalk[3]:
                print("danger, pe trecere")
            elif crosswalk[1] < person[3] + 15 or crosswalk[3] > person[3] - 15:
                if person[0] < crosswalk[2] + 350:
                    # Get orientation :
                    landmarks = pose_estimator.get_pose_landmarks(np.asarray(source)[persons[0][1]:persons[0][3], persons[0][0]:persons[0][2]])
                    hips_label = classify_by_hip_distance(landmarks)
                    if hips_label == "side":
                        print("warning, pe langa trecere")
                    else:
                        print("langa trecere, dar nu se uita in stanga sau dreapta")
            else:
                print("nici un pericol")

crosswalk 0:
danger, pe trecere
crosswalk 1:
danger, pe trecere
